# Домашнее задание 1 (26 pts)


- (2 pts) Докажите, что $\| A \|_F \le \sqrt{\mathrm{rank}(A)} \| A \|_2$.\
  $\| A \|^2_F = \sum \limits_{i = 1}^{\mathrm{rank}(A)} \sigma_i^2 \leq \mathrm{rank}{(A)} \max \limits_{i} \sigma_i^2 = \mathrm{rank}{(A)} \| A \|^2_2 \Rightarrow \| A \|_F \le \sqrt{\mathrm{rank}(A)} \| A \|_2$
- (2 pts) Покажите, что для любых $m, n$ и $k \le \min(m, n)$ существует $A \in \mathbb{R}^{m \times n}: \mathrm{rank}(A) = k$, такая что $\| A \|_F = \sqrt{\mathrm{rank}(A)} \| A \|_2$.\
   Возьмем матрицу:
  $
    A = \begin{bmatrix}
    I_k & 0 \\
    0 & 0
    \end{bmatrix}_{m\times n}
$\
   \
   $\| A \|_F = \sqrt{k} = \sqrt{\mathrm{rank}{(A)}} * 1 = \sqrt{\mathrm{rank}{(A)}} * \max \limits_{i} \sigma_i = \sqrt{\mathrm{rank}{(A)}} \| A \|_2$
- (2 pts) Докажите, что $\| A B \|_F \le \| A \|_2 \| B \|_F$.\
  $\| AB \|^2_F = \| A [b_1, \dots, b_n] \|^2_F = \| [Ab_1, \dots, Ab_n] \|^2_F = \sum \limits_{i = 1}^n \| Ab_i \|^2_2 \leq \| A \|^2_2 \sum \limits_{i = 1}^n \| b_i \|^2_2 = \| A \|^2_2 \| B \|^2_F \Rightarrow$\
  $\| A B \|_F \leq \| A \|_2 \| B \|_F$

- (2 pts) Пусть матрица $A \in \mathbb{C}^{n \times n}$ и её сингулярное разложение $A = U\Sigma V^*$. Найдите сингулярное разложение матрицы $\begin{bmatrix} 0 & A^* \\ A & 0 \end{bmatrix}$ размера $2n \times 2n$\
   \
   Сделаем следующие преобразования:\
   \
   $
\begin{bmatrix}
0 & A^* \\
A & 0
\end{bmatrix}
=
\begin{bmatrix}
0 & V\Sigma U^* \\
U\Sigma V^* & 0
\end{bmatrix}
=
\begin{bmatrix}
0 & V \\
U & 0
\end{bmatrix}
\begin{bmatrix}
\Sigma & 0
\\ 0 & \Sigma
\end{bmatrix}
\begin{bmatrix}
V^* & 0
\\ 0 & U^*
\end{bmatrix}
$\
   \
   Это сингулярное разложение, потому что матрицы слева и справа очевидно унитарные
- (7 pts) Пусть известно QR разложение прямоугольной $m\times n, m > n$ матрицы $A = QR$. Покажите, как изменится данное разложение при следующих изменениях матрицы $A$
  - замена $k$-го столбца на другой вектор-столбец
  - конкатенация новой строки - новая матрица размерности $(m+1) \times n$
  - конкатенация нового столбца - новая матрица размерности $m \times n+1$

Оцените сложность каждого из этих преобразований. Реализуйте все три преобразования и покажите численно, что ваши алгоритмы обновлений работают корректно и быстрее наивных реализаций с полным пересчётом матриц $Q$ и $R$. Используйте тестовые матрицы размерностями не меньше нескольких сотен строк и столбцов.


In [39]:
import numpy as np

np.random.seed(42)


def givens_rotation(Q, R, k, l, column):
    a_k, a_l = R[k, column], R[l, column]
    cos = a_k / (a_k**2 + a_l**2) ** 0.5
    sin = -a_l / (a_k**2 + a_l**2) ** 0.5
    G = np.array([[cos, -sin], [sin, cos]])
    Q[:, k : l + 1] = Q[:, k : l + 1] @ G.T
    R[k : l + 1] = G @ R[k : l + 1]


M, N, k = 2000, 1000, 0

Посмотрим сколько делается QR разложение


In [40]:
A = np.random.rand(M, N)

In [41]:
%%timeit

Q, R = np.linalg.qr(A, mode="complete")

111 ms ± 11 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 1


Обозначим новый столбец за $u$, тогда заменим k-ый столбец в матрице $R$ на $Q^*u$. Тогда с помощью преобразований Гивенса мы можем обнулить все элементы ниже диагонали в k-ом столбце за $O(M^2)$, при этом получится матрица верхне-гессенберговая. Её же можно привести к верхнетреугольной за $O(MN)$. Преобразование Гивенса не портит ортогональность, поэтому $Q$ остаётся унитарной матрицей. Итоговая асимптотика $O(M^2)$


In [42]:
A = np.random.rand(M, N)
Q, R = np.linalg.qr(A, mode="complete")
Q.shape, R.shape

((2000, 2000), (2000, 1000))

Сгенерируем новый столбец


In [43]:
u = np.random.rand(M, 1)
u.shape

(2000, 1)

Заменим столбец в матрице $R$ и обнулим все что ниже диагонали с помощью преобразований Гивенса и узнаем сколько это делается


In [44]:
%%timeit

R[:, k] = (Q.T @ u)[:, 0]
for row in range(M - 1, k, -1):  # O(M^2)
    givens_rotation(Q, R, row - 1, row, k)  # O(M)
R[k + 1 :, k] = 0  # для наглядности
for column in range(k + 1, N - 1):  # O(MN)
    givens_rotation(Q, R, column, column + 1, column)  # O(M)
    R[column + 1 :, column] = 0  # для наглядности

81.5 ms ± 337 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
R[:, k] = (Q.T @ u)[:, 0]
for row in range(M - 1, k, -1):  # O(M^2)
    givens_rotation(Q, R, row - 1, row, k)  # O(M)
R[k + 1 :, k] = 0  # для наглядности
for column in range(k + 1, N - 1):  # O(MN)
    givens_rotation(Q, R, column, column + 1, column)  # O(M)
    R[column + 1 :, column] = 0  # для наглядности
R[:10, :5]

array([[25.53335917, 19.27525813, 19.43577843, 19.50079717, 19.48112715],
       [ 0.        , 16.91215537,  7.53449417,  6.77670583,  7.24143148],
       [ 0.        ,  0.        , 15.4126777 ,  4.39788857,  4.84651498],
       [ 0.        ,  0.        ,  0.        , 14.66298051,  3.302877  ],
       [ 0.        ,  0.        ,  0.        ,  0.        , 14.23568398],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

Проверим корректность


In [45]:
A[:, k] = u[:, 0]
np.isclose(A, Q @ R).all()

True

Как видно мы получили QR разложение для новой матрицы $A$, потратив на это $O(M^2)$ операций. Тесты скорости это подтверждают, даже учитывая, что рассмотрен самый худщий случай замены 1 столбца и были использованы питоновские циклы.


### 2


Рассмотрим преобразование:\
\
$
\begin{bmatrix}
A \\
u^*
\end{bmatrix} =
\begin{bmatrix}
QR \\
u^*
\end{bmatrix} =
\begin{bmatrix}
0 & Q \\
1 & 0
\end{bmatrix}
\begin{bmatrix}
u^* \\
R
\end{bmatrix}
$\
\
Отсюда видно, что матрица слева очевидно ортогональная, и что матрица справа верхне-гессенберговая. Поэтому как и предидущем пункте её можно сделать верхнетреугольной с помощью преобразований Гивенса.


In [51]:
A = np.random.rand(M, N)
Q, R = np.linalg.qr(A, mode="complete")
Q.shape, R.shape

((2000, 2000), (2000, 1000))

Сгенерируем новую строчку


In [52]:
u = np.random.rand(1, N)
u.shape

(1, 1000)

Построим новые матрицы $Q$ и $R$


In [53]:
R = np.vstack((u, R))
Q_ = np.zeros((M + 1, M + 1))
Q_[:-1, 1:] = Q
Q_[M, 0] = 1
Q = Q_
Q.shape, R.shape

((2001, 2001), (2001, 1000))

In [54]:
%%timeit


for column in range(N):  # O(MN)
    givens_rotation(Q, R, column, column + 1, column)  # O(M)
    R[column + 1, column] = 0  # для наглядности

18 ms ± 397 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [49]:
for column in range(N):  # O(MN)
    givens_rotation(Q, R, column, column + 1, column)  # O(M)
    R[column + 1, column] = 0  # для наглядности
R[:10, :5]

array([[25.88998781, 19.15261435, 19.92530329, 19.66584846, 19.13928432],
       [ 0.        , 17.59110229,  7.33702939,  7.82759444,  7.30712208],
       [ 0.        ,  0.        , 15.01904284,  4.51790694,  4.65105228],
       [ 0.        ,  0.        ,  0.        , 14.74042069,  3.3119909 ],
       [ 0.        ,  0.        ,  0.        ,  0.        , 14.15388739],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

Проверим корректность


In [50]:
A = np.vstack((A, u))
np.isclose(A, Q @ R).all()

True

Как видно мы получили QR разложение для новой матрицы $A$, потратив на это $O(MN)$ операций. Тесты скорости это подтверждают.


### 3


Этот случай является частным случаем пункта 1. Но его можно сделать по другому, не используя преобразование Гивенса.\
Возьмем сокращенное QR разложение. Сделаем проекцию вектора $u$ на столбцы матрицы $Q$: $QQ^*u$. Тогда вектор $u - QQ^*u$ будет ортогонален всем столбцам матрицы $Q$, поэтому его можно включить в неё, заранее нормировав. Тогда в матрице $R$ нужно будет добавить $Q^*u$.


In [104]:
A = np.random.rand(M, N)
Q, R = np.linalg.qr(A)
Q.shape, R.shape

((2000, 1000), (1000, 1000))

Сгенерируем новый столбец


In [105]:
u = np.random.rand(M, 1)
u.shape

(2000, 1)

Применим преобразования, описанные ранее


In [102]:
%%timeit


u_coeffs = Q.T @ u  # O(MN)
q = u - Q @ u_coeffs  # O(MN)
q_norm = np.linalg.norm(q)
q /= q_norm

94.7 µs ± 2.53 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [106]:
u_coeffs = Q.T @ u  # O(MN)
q = u - Q @ u_coeffs  # O(MN)
q_norm = np.linalg.norm(q)
q /= q_norm
Q = np.hstack((Q, q))
R_ = np.zeros((N + 1, N + 1))
R_[:-1, :-1] = R
R_[:-1, -1] = u_coeffs[:, 0]
R_[-1, -1] = q_norm
R = R_

Проверим корректность


In [107]:
A = np.hstack((A, u))
np.isclose(A, Q @ R).all()

True

Как видим всё корректно. И мы построили новое разложение за $O(MN)$.


###

- (3 pts) Пусть $A$ такая матрица, что $a_{i,j} \geq 0$ и $\sum_{j}a_{i,j} = 1$ (сумма элементов вкаждой строке равна 1). Докажите, что $A$ имеет собственное значение $\lambda=1$ и что все собственные значения $\lambda_i$ таковы что $|\lambda_i| \le 1$

  $|A - \lambda I| =
\begin{vmatrix}
a_{11} - \lambda & \dots & a_{1n} \\
\dots & \dots & \dots \\
a_{n1} & \dots & a_{nn} - \lambda
\end{vmatrix}_{n\times n}
=
\begin{vmatrix}
a_{11} - \lambda & \dots & 1 - \lambda \\
\dots & \dots & \dots \\
a_{n1} & \dots & 1 - \lambda
\end{vmatrix}_{n\times n}
\Rightarrow \lambda = 1
$ - собственное значение

  $\rho (A) \leq \| A \|$ для любой матричной нормы (в частности для бесконечной)\
   $\rho (A) \leq \| A \|_\infty = \max \limits_{1 \leq i \leq n} \sum \limits_{j = 1}^n |a_{ij}| = \max \limits_{1 \leq i \leq n} \sum \limits_{j = 1}^n a_{ij} = 1 \Rightarrow |\lambda_i| \leq 1$

- (3 pts) Докажите, что нормальная матрица Эрмитова iff её собственные значения действительны. Докажите, что нормальная матрица унитарна iff все её собственные значения таковы что $|\lambda| = 1$.

  Если матрица нормальная:

  $A = U\Lambda U^* = U\Lambda^* U^* = A^* \Leftrightarrow \Lambda = \Lambda^*$\
  $AA^* = U\Lambda U^*U\Lambda^* U^* = U\Lambda\Lambda^* U^* = I, A^*A = U\Lambda^* U^*U\Lambda U^* = U\Lambda^*\Lambda U^* = I \Leftrightarrow \Lambda^*\Lambda = \Lambda\Lambda^* = I$

- (5 pts) Найдите аналитическое выражение для собственнных значений возмущённого Жорданова блока

  $$
  J(\varepsilon) =
      \begin{bmatrix}
       \lambda & 1 & & & 0 \\
      0 & \lambda & 1 & & \\
       0 & 0 & \ddots & \ddots & \\
       \ldots & \ldots & \ldots & \lambda & 1 \\
       \varepsilon & 0 & 0 & 0 & \lambda  \\
      \end{bmatrix}_{n\times n}
  $$

  $
  |J(\varepsilon) - xI| =
  \begin{vmatrix}
  \lambda - x & 1 & & & 0 \\
  0 & \lambda - x & 1 & & \\
  0 & 0 & \ddots & \ddots & \\
  \ldots & \ldots & \ldots & \lambda - x & 1 \\
  \varepsilon & 0 & 0 & 0 & \lambda - x \\
  \end{vmatrix}_{n\times n} =
  (\lambda - x)
  \begin{vmatrix}
  \lambda - x & 1 & & \\
  0 & \ddots & \ddots & \\
  \ldots & \ldots & \lambda - x & 1 \\
  0 & 0 & 0 & \lambda - x \\
  \end{vmatrix}_{n\times n} +
  (-1) ^ {n - 1} \varepsilon
  \begin{vmatrix}
  1 & & & 0 \\
  \lambda - x & 1 & & \\
  0 & \ddots & \ddots & \\
  \ldots & \ldots & \lambda - x & 1 \\
  \end{vmatrix}\_{n\times n} =
  (\lambda - x)^n + (-1) ^ {n - 1} \varepsilon = 0
  $

  $(\lambda - x)^n = (-1)^n \varepsilon$

  $|\lambda - x|^n = |\varepsilon|$

  $|\lambda - x| = |\varepsilon|^{\frac{1}{n}} \rightarrow 1$

  Прокомментируйте как собственные значения $J(0)$ возмущены для больших $n$? Что это говорит об устойчивости получения спектра матрицы через Жорданов блок?

  Для больших $n$ собственные значения стремятся изменится по модулю на $1$. Из этого мы можем сделать вывод, что считать спектр матрицы через ЖНФ не самая лучшая идея, так как эти вычисления неустойчивы.
